# Imports

In [384]:
# Usual stuff
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

# Typing
from typing import Callable

In [385]:
# Sympy pretty printing
sp.init_printing()

# Hamiltonian

We construct a matrix for the following Hamiltonian:

$$
\begin{align*}

H^\text{MF} &= \sum_{\textbf k > 0} H_{\textbf k}

\end{align*}
$$

Where:

- $H_\textbf k$ couples only the $\pm\textbf k$ modes:
    
    $$
    \begin{align*}
    
    H_{\textbf k} &= \xi_\textbf k(\hat n_\textbf k + \hat n_{-\textbf k}) + U(\hat n_{\textbf k\uarr}\hat n_{\textbf k\darr} + \hat n_{-\textbf k\uarr}\hat n_{-\textbf k \darr})
    \\[0.2cm]
    
    &\quad +\Delta^*(b_\textbf k + b_{-\textbf k}) + \Delta(b_\textbf k^\dag + b_{-\textbf k}^\dag)
    
    \end{align*}
    $$
    
- $\Delta$ has to be solved self-consistently for:
    
    $$
    \begin{align*}
    
    \Delta &= -g\sum_\textbf k\lang b_\textbf k\rang
    
    \end{align*}
    $$

- $b_\textbf k$ is the pair operator for $s$-wave coupling:
    
    $$
    \begin{align*}
    
    b_\textbf k &= c_{-\textbf k\darr}c_{\textbf k\uarr}
    \\[0.2cm]

    b_\textbf k^\dag &= c_{\textbf k\uarr}^\dag c_{-\textbf k\darr}^\dag
    
    \end{align*}
    $$

We write the Hamiltonian as follows, were the states can be read as binary numbers:
$$
\begin{align*}

H_{\textbf k} &= \sum_{n_{\pm\textbf k, \pm\sigma} = 0}^1\sum_{n'_{\pm\textbf k, \pm\sigma} = 0}^1 \lang n_{\textbf k \uarr}n_{\textbf k \darr}n_{-\textbf k \uarr}n_{-\textbf k \darr} \mid H_{\textbf k}  \mid n'_{\textbf k \uarr}n'_{\textbf k \darr}n'_{-\textbf k \uarr}n'_{-\textbf k \darr} \rang 
\ket{n_{\textbf k \uarr}n_{\textbf k \darr}n_{-\textbf k \uarr}n_{-\textbf k \darr}} \bra{n'_{\textbf k \uarr}n'_{\textbf k \darr}n'_{-\textbf k \uarr}n'_{-\textbf k \darr} }

\end{align*}
$$

In [ ]:
def eps(kk: np.ndarray) -> np.ndarray:
    """
        Compute the dispersion relation for a given set of vectors `kk`.
    """
    return -2 * np.sum(np.cos(kk), axis = 0)


def fock(decimal: int) -> list[int]:
    string = f"{bin(decimal)[2:]:0>4}"
    nks = list(map(int, list(string)))
    return nks


def bpair():
    """
        Create the matrix representation for bk.
    """

    # Dimension of the Fock space
    dim = 16
    
    # Build the operator
    bk = np.zeros((dim, dim), dtype=np.float64)

    # Matrix elements
    for i in range(dim):
        for j in range(dim):
            npkui, npkdi, nmkui, nmkdi = fock(i)
            npkuj, npkdj, nmkuj, nmkdj = fock(j)

            bk[i, j] += (-1)**(npkuj - 1 + npkdj + nmkuj) * ((npkui + 1) == npkuj) * ((nmkdi + 1) == nmkdj) * (npkdi == npkdj) * (nmkui == nmkuj)
    
    return bk


def ntotal():
    """
        Create the matrix representation for ntotal.
    """

    # Dimension of the Fock space
    dim = 16
    
    # Build the operator
    nk = np.zeros((dim, dim), dtype=np.float64)

    # Matrix elements
    for i in range(dim):
        npku, npkd, nmku, nmkd = fock(i)
        nk[i, i] += npku + npkd + nmku + nmkd

    return nk


def ham(kk: np.ndarray, U: float, mu: float, Delta: complex):
    """
        Create the Hamiltonian Hk numerically.
    """

    # Dimension of the Fock space
    dim = 16

    # Number of k-points
    Nk = len(kk[0])

    # Dispersion relation
    xi = eps(kk) - mu

    # Build the Hamiltonian
    Hk = np.zeros((Nk, dim, dim), dtype=np.float64)
    
    # Diagonal terms
    for i in range(dim):
        npku, npkd, nmku, nmkd = fock(i)
        Hk[:, i, i] += xi * (npku + npkd + nmku + nmkd)
        Hk[:, i, i] += U * (npku * npkd + nmku * nmkd)
    
    # Off-diagonal terms, taking into account the phase factor!
    # I will do only the destruction terms, as the creation ones are obtained by conjugation.
    for i in range(dim):
        for j in range(dim):
            npkui, npkdi, nmkui, nmkdi = fock(i)
            npkuj, npkdj, nmkuj, nmkdj = fock(j)

            Hk[:, i, j] += Delta.conjugate() * ((-1)**(npkuj - 1 + npkdj + nmkuj) * ((npkui + 1) == npkuj) * ((nmkdi + 1) == nmkdj) * (npkdi == npkdj) * (nmkui == nmkuj) + 
                                                (-1)**(npkdj) * ((nmkui + 1) == nmkuj) * ((npkdi + 1) == npkdj) * (npkui == npkuj) * (nmkdi == nmkdj))
    
    # Hermitian conjugate
    for i in range(dim):
        for j in range(dim):
            if i != j and Hk[0, j, i] == 0:
                Hk[:, j, i] += np.conjugate(Hk[:, i, j])
    
    return Hk


def solve(L: int, d: int, U: float, mu: float, g: float, T: float, delta_start: float = 0.001, delta_eps: float = 1):
    """
        Solve the mean-field Hamiltonian for `L` k-points along each `d` directions for temperature `T` and chemical potential `mu`.

        Self-consistently compute Delta starting at `delta_start` with an error of `delta_eps` / (L**d).
    """

    # Dimension of the Fock space
    dim = 16

    # Possible adjustments
    # Take out the [:-1] from kk
    # Change the kx > 0 into kx >= 0


    # OBTAINING K-POINTS

    # Determine values of k in our lattice
    kk = np.linspace(-np.pi, np.pi, L + 1)[:-1]

    # Make a mesh of k values
    grid_kk = np.meshgrid(*[kk]*d, indexing='ij')

    # Convert to a single array with shape (d, L ** d)
    mesh_kk = np.stack(grid_kk).reshape(d, len(kk) ** d)

    # Select only kx > 0
    mesh_kk = mesh_kk[:, mesh_kk[0] > 0]

    # Number of k-points
    Nk = len(mesh_kk[0])

    

    # OPERATORS TO AVERAGE

    # Matrix for b
    bk = bpair()

    # Matrix for n
    nt = ntotal()


    # SELF-CONSISTENT CALCULATION OF DELTA

    delta_error = delta_eps + 1
    while delta_error > delta_eps / (L**d):

        # Get the Hamiltonians, which has size (Nk, dim, dim)
        Hk = ham(mesh_kk, U, mu, delta_start)

        # Debug: Print the Hamiltonian
        # with np.printoptions(precision=3, suppress=True, threshold=100_000, linewidth=200):
        #     print(Hk)
        #     print(f"This is Hermitian! {(Hk == Hk.conj().transpose(0, 2, 1)).all()}")

        # Diagonalize Hamiltonians
        vals, vecs = np.linalg.eigh(Hk)

        # Minimum energy, to scale Boltzmann factors
        E0 = np.min(vals, axis=1)
        E0 = np.tile(E0[:, None], (1, dim))
        vals = vals - E0

        # Compute partition function
        Z = np.sum(np.exp(-vals/T), axis = 1)
        
        # Compute Delta via a trace
        delta_new = 0
        for ik in range(dim):
            # Build bra and ket
            bra = vecs[:, :, ik]
            ket = bra.T
            bra = np.conjugate(ket)
            
            # Compute average (sum over axis=0 is expectation value, sum over axis=1 is over various k-points)
            delta_new += np.sum(np.exp(-vals[:, ik]/T) * bra * (bk @ ket) / Z)
        
        delta_new *= -g

        # Compute error
        delta_error = np.abs(delta_start - delta_new)

        # Prepare next loop
        delta_start = delta_new
    

    # SOLVE FOR CONVERGED DELTA

    # Get the Hamiltonian, which has size (Nk, dim, dim)
    Hk = ham(mesh_kk, U, mu, delta_start)
    
    # Diagonalize Hamiltonians
    vals, vecs = np.linalg.eig(Hk)
    
    # Minimum energy, to scale Boltzmann factors
    E0 = np.min(vals, axis=1)
    E0 = np.tile(E0[:, None], (1, dim))
    vals = vals - E0

    # Compute partition function
    Z = np.sum(np.exp(-vals/T), axis = 1)

    # Debug: See the eigenstates
    # with np.printoptions(precision=3, suppress=True, threshold=100_000, linewidth=200):
    #     print(vecs)
    #     print(vals)
    #     print(np.exp(-vals/T))

    

    # Compute average number of particles
    Ne = 0
    for ik in range(dim):

        # Build bra and ket
        bra = vecs[:, :, ik]
        ket = bra.T
        bra = ket.conj()
        
        # Compute average (sum over axis=0 is expectation value, sum over axis=1 is over various k-points)
        Ne += np.sum(np.exp(-vals[:, ik]/T) * bra * (nt @ ket) / Z)
    
    n = Ne / (2*Nk)

    # Compute average kinetic energy along the x direction
    Kxx = 0
    for ik in range(dim):
        # Build bra and ket
        bra = vecs[:, :, ik]
        ket = bra.T
        bra = ket.conj()

        # Second derivative of eps along kx
        epsxx = 2 * np.cos(mesh_kk[0, :])
        
        # Compute average (sum over axis=0 is expectation value, sum over axis=1 is over various k-points)
        Kxx += np.sum(np.exp(-vals[:, ik]/T) * epsxx * bra * (nt @ ket) / Z)
    
    Kxx = np.pi / (L ** d) * Kxx

    return delta_start, n, Kxx

In [387]:
# PARAMETERS
L = 1000
d = 1
U = 1
mu = 0.5
g = 0
T = 0.01

# Example CALCULATION
Delta, n, Kxx = solve(L, d, U, mu, g, T)

## Average Occupation

In [388]:
def occupations(mesh_xk: np.ndarray, U: float, beta: float, cutoff: float = 30):
    """
        Compute the average occupation of each state.
    """

    # Omega regions
    mesh_nk = np.zeros_like(mesh_xk)

    # Arguments of exponentials
    arg_zk = -beta * mesh_xk
    arg_zk1U = -beta * (mesh_xk + U)
    arg_zk2U = -beta * (2 * mesh_xk + U)

    # Determine when the approximate formula is required
    approx = arg_zk > cutoff

    # Clip to prevent overflow errors
    arg_zk = np.clip(arg_zk, -700, 700)
    arg_zk1U = np.clip(arg_zk1U, -700, 700)
    arg_zk2U = np.clip(arg_zk2U, -700, 700)

    # Use the approximate formula
    mesh_nk[approx] = 1 - 1 / (2 + np.exp(arg_zk1U[approx]))

    # Compute log(1 - nk)
    mesh_zk = np.exp(arg_zk)
    mesh_nk[~approx] = np.log1p(mesh_zk[~approx]) - np.log1p(2 * mesh_zk[~approx] + np.exp(arg_zk2U[~approx]))

    # Compute nk
    mesh_nk[~approx] = 1 - np.exp(mesh_nk[~approx])

    return mesh_nk

## Previous Attempt - Hamiltonian with SymPy

In [389]:
'''
# We are using the notation npku as "n plus k up"
def ham():
    """
        Create the Hamiltonian Hk symbolically.
    """

    # Dimension of the Fock space
    dim = 16

    # Build the Hamiltonian
    Hk = sp.zeros(dim, dim)

    # Symbols
    xi, U, Delta = sp.symbols("xi, U, Delta")
    Delta_star = sp.conjugate(Delta)
    
    # Diagonal terms
    for i in range(dim):
        npku, npkd, nmku, nmkd = fock(i)
        Hk[i, i] += xi * (npku + npkd + nmku + nmkd)
        Hk[i, i] += U * (npku * npkd + nmku * nmkd)
    
    # Off-diagonal terms, taking into account the phase factor!
    # I will do only the destruction terms, as the creation ones are obtained by conjugation.
    for i in range(dim):
        for j in range(dim):
            npkui, npkdi, nmkui, nmkdi = fock(i)
            npkuj, npkdj, nmkuj, nmkdj = fock(j)

            Hk[i, j] += Delta_star * ((-1)**(npkuj - 1 + npkdj + nmkuj) * ((npkui + 1) == npkuj) * ((nmkdi + 1) == nmkdj) * (npkdi == npkdj) * (nmkui == nmkuj) + 
                                      (-1)**(npkdj) * ((nmkui + 1) == nmkuj) * ((npkdi + 1) == npkdj) * (npkui == npkuj) * (nmkdi == nmkdj))
    
    # Hermitian conjugate
    for i in range(dim):
        for j in range(dim):
            if i != j and Hk[j, i] == 0:
                Hk[j, i] += sp.conjugate(Hk[i, j])
    
    return Hk


def solve(N: int, d: int, U: float, mu: float):
    """
        Solve the mean-field Hamiltonian for N k-points along each d directions.
    """

    # Get the Hamiltonian
    Hk = ham()

    print("Hamiltonian:")
    print(sp.latex(Hk))

    # Diagonalize the Hamiltonian
    (vecs, vals) = Hk.diagonalize(sort=True, normalize=True)

    print("\nEigenvectors:")
    print(sp.latex(vecs))
    print(sp.latex(vals))

    print("\nEigenvectors in string form:")
    print(str(vecs))
    print(str(vals))

'''

'\n# We are using the notation npku as "n plus k up"\ndef ham():\n    """\n        Create the Hamiltonian Hk symbolically.\n    """\n\n    # Dimension of the Fock space\n    dim = 16\n\n    # Build the Hamiltonian\n    Hk = sp.zeros(dim, dim)\n\n    # Symbols\n    xi, U, Delta = sp.symbols("xi, U, Delta")\n    Delta_star = sp.conjugate(Delta)\n    \n    # Diagonal terms\n    for i in range(dim):\n        npku, npkd, nmku, nmkd = fock(i)\n        Hk[i, i] += xi * (npku + npkd + nmku + nmkd)\n        Hk[i, i] += U * (npku * npkd + nmku * nmkd)\n    \n    # Off-diagonal terms, taking into account the phase factor!\n    # I will do only the destruction terms, as the creation ones are obtained by conjugation.\n    for i in range(dim):\n        for j in range(dim):\n            npkui, npkdi, nmkui, nmkdi = fock(i)\n            npkuj, npkdj, nmkuj, nmkdj = fock(j)\n\n            Hk[i, j] += Delta_star * ((-1)**(npkuj - 1 + npkdj + nmkuj) * ((npkui + 1) == npkuj) * ((nmkdi + 1) == nmkdj) * (np

Result:

$$
\left[\begin{array}{cccccccccccccccc}0 & 0 & 0 & 0 & 0 & 0 & - \overline{\Delta} & 0 & 0 & \overline{\Delta} & 0 & 0 & 0 & 0 & 0 & 0\\0 & \xi & 0 & 0 & 0 & 0 & 0 & - \overline{\Delta} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & \xi & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & - \overline{\Delta} & 0 & 0 & 0 & 0\\0 & 0 & 0 & U + 2 \xi & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & \xi & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & - \overline{\Delta} & 0 & 0\\0 & 0 & 0 & 0 & 0 & 2 \xi & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\- \Delta & 0 & 0 & 0 & 0 & 0 & 2 \xi & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \overline{\Delta}\\0 & - \Delta & 0 & 0 & 0 & 0 & 0 & U + 3 \xi & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \xi & 0 & 0 & 0 & 0 & 0 & - \overline{\Delta} & 0\\\Delta & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 2 \xi & 0 & 0 & 0 & 0 & 0 & - \overline{\Delta}\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 2 \xi & 0 & 0 & 0 & 0 & 0\\0 & 0 & - \Delta & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & U + 3 \xi & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & U + 2 \xi & 0 & 0 & 0\\0 & 0 & 0 & 0 & - \Delta & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & U + 3 \xi & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & - \Delta & 0 & 0 & 0 & 0 & 0 & U + 3 \xi & 0\\0 & 0 & 0 & 0 & 0 & 0 & \Delta & 0 & 0 & - \Delta & 0 & 0 & 0 & 0 & 0 & 2 U + 4 \xi\end{array}\right]
$$